In [1]:
from huggingface_hub import hf_hub_download

path = hf_hub_download(repo_id="facebook/esmfold_v1", filename="pytorch_model.bin")
print("Model weight path:", path)

Model weight path: C:\Users\mkhsi\.cache\huggingface\hub\models--facebook--esmfold_v1\snapshots\75a3841ee059df2bf4d56688166c8fb459ddd97a\pytorch_model.bin


In [2]:
from transformers import EsmForProteinFolding
model = EsmForProteinFolding.from_pretrained(r"C:\Users\mkhsi\.cache\huggingface\hub\models--facebook--esmfold_v1\snapshots\75a3841ee059df2bf4d56688166c8fb459ddd97a", low_cpu_mem_usage=True)
#model = EsmForProteinFolding.from_pretrained("esmfold_v1", low_cpu_mem_usage=True)
print(model.name_or_path)

Some weights of the model checkpoint at C:\Users\mkhsi\.cache\huggingface\hub\models--facebook--esmfold_v1\snapshots\75a3841ee059df2bf4d56688166c8fb459ddd97a were not used when initializing EsmForProteinFolding: ['esm.embeddings.position_embeddings.weight']
- This IS expected if you are initializing EsmForProteinFolding from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing EsmForProteinFolding from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of EsmForProteinFolding were not initialized from the model checkpoint at C:\Users\mkhsi\.cache\huggingface\hub\models--facebook--esmfold_v1\snapshots\75a3841ee059df2bf4d56688166c8fb459ddd97a and are newly initialized: ['esm.contact_head.regression.bias'

C:\Users\mkhsi\.cache\huggingface\hub\models--facebook--esmfold_v1\snapshots\75a3841ee059df2bf4d56688166c8fb459ddd97a


In [3]:
import torch
# === CONFIGURATION ===
device_name = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device(device_name)

# Move model to GPU (with optimizations)
if device_name == "cuda":
    gpu_ram_size = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"GPU has {gpu_ram_size:.2f} GB of RAM")

    model.esm = model.esm.half()  # use half-precision for speed and memory
    torch.backends.cuda.matmul.allow_tf32 = True  # enable TF32 for speed (optional)

    if gpu_ram_size < 16:
        model.trunk.set_chunk_size(8)  # reduce chunk size if VRAM is limited

model = model.to(device)
model.eval()

C:\Users\mkhsi\test\Lib\site-packages\torch\cuda\__init__.py:174: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\c10\cuda\CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


EsmForProteinFolding(
  (esm): EsmModel(
    (embeddings): EsmEmbeddings(
      (word_embeddings): Embedding(33, 2560, padding_idx=1)
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): EsmEncoder(
      (layer): ModuleList(
        (0-35): 36 x EsmLayer(
          (attention): EsmAttention(
            (self): EsmSelfAttention(
              (query): Linear(in_features=2560, out_features=2560, bias=True)
              (key): Linear(in_features=2560, out_features=2560, bias=True)
              (value): Linear(in_features=2560, out_features=2560, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
              (rotary_embeddings): RotaryEmbedding()
            )
            (output): EsmSelfOutput(
              (dense): Linear(in_features=2560, out_features=2560, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (LayerNorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
          )
          (intermediate

In [4]:
def read_fasta(
    path,
    keep_gaps=True,
    keep_insertions=True,
    to_upper=False,
):
    with open(path, "r") as f:
        for result in read_alignment_lines(
            f, keep_gaps=keep_gaps, keep_insertions=keep_insertions, to_upper=to_upper
        ):
            yield result


def read_alignment_lines(
    lines,
    keep_gaps=True,
    keep_insertions=True,
    to_upper=False,
):
    seq = desc = None

    def parse(s):
        if not keep_gaps:
            s = re.sub("-", "", s)
        if not keep_insertions:
            s = re.sub("[a-z]", "", s)
        return s.upper() if to_upper else s

    for line in lines:
        # Line may be empty if seq % file_line_width == 0
        if len(line) > 0 and line[0] == ">":
            if seq is not None:
                yield desc, parse(seq)
            desc = line.strip().lstrip(">")
            seq = ""
        else:
            assert isinstance(seq, str)
            seq += line.strip()
    assert isinstance(seq, str) and isinstance(desc, str)
    yield desc, parse(seq)


In [5]:
from pathlib import Path
import time
def generate_pdb(model, fastafile, outputfolder):
    start = time.time()
    # Read fasta and sort sequences by length
    Path(outputfolder).mkdir(exist_ok=True)
    #os.mkdir(args.outfolder, exist_ok=True)
    all_sequences = sorted(read_fasta(fastafile), key=lambda header_seq: len(header_seq[1]))[0:1]
    #logger.info(f"Loaded {len(all_sequences)} sequences from {args.fasta}")
    print("Loaded ",len(all_sequences), " sequences from ",fastafile)

    ### generate pdb over all samples
    #for samplelabel, sampleseq in all_sequences[0]:
    for i in range(len(all_sequences)):
        samplelabel=all_sequences[i][0]
        sampleseq=all_sequences[i][1]
        #samplelabel = samplelabel.split('|')[0] if ':1|' in samplelabel else [samplelabel]
        samplelabel = samplelabel.split('|')[1]

        #print(samplelabel, sampleseq, len(sampleseq))
        print(samplelabel)
        print(sampleseq, len(sampleseq))

        with torch.no_grad():
            #output = model(tokenized_input)
            output = model.infer_pdb(sampleseq)
            #print(output)
        #pdb_alignment(pdb_code,output)
        ## alignment to origial PDB for assemble with other molecules

        with open(outputfolder+'/'+samplelabel+".pdb", "w") as f:
            f.write(output)
        end = time.time()
        print(f'{outputfolder}/{samplelabel}.pdb was generated and saved, execution time: {end - start:.3f} seconds.')



In [6]:
outputfolder='tkd_variants_pdb'
#fastafile='mutated_tkd_proteins.fasta'
fastafile='C797S_mutated_tkd_proteins.fasta'
generate_pdb(model, fastafile, outputfolder)


Loaded  1  sequences from  C797S_mutated_tkd_proteins.fasta
p.Cys797Ser
FKKIKVLGSGAFGTVYKGLWIPEGEKVKIPVAIKELREATSPKANKEILDEAYVMASVDNPHVCRLLGICLTSTVQLITQLMPFGSLLDYVREHKDNIGSQYLLNWCVQIAKGMNYLEDRRLVHRDLAARNVLVKTPQHVKITDFGLAKLLGAEEKEYHAEGGKVPIKWMALESILHRIYTHQSDVWSYGVTVWELMTFGSKPYDGIPASEISSILEKGERLPQPPICTIDVYMIMVKCWMIDADSRPKFRELIIEFSKMARDPQRY 267
tkd_variants_pdb/p.Cys797Ser.pdb was generated and saved, execution time: 390.199 seconds.


In [7]:
import py3Dmol
with open("tkd_variants_pdb/p.Cys797Ser.pdb", "r") as f:
    pdb_data = f.read()

view = py3Dmol.view(width=600, height=400)
view.addModel(pdb_data, "pdb")
view.setStyle({"cartoon": {"color": "spectrum"}})
view.zoomTo()
view.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [8]:
import requests
pdb_id="4I22"
def download_pdb_from_rcsb(pdb_id, save_path=None):
    url = f"https://files.rcsb.org/download/{pdb_id.upper()}.pdb"
    response = requests.get(url)
    if response.status_code == 200:
        if save_path is None:
            save_path = f"{pdb_id}.pdb"
        with open(save_path, "w") as f:
            f.write(response.text)
        print(f"PDB file saved to: {save_path}")
    else:
        print(f"Failed to download PDB {pdb_id}: HTTP {response.status_code}")



In [9]:
pdb_id="4I22"
download_pdb_from_rcsb(pdb_id)
with open(f'{pdb_id}.pdb', "r") as f:
    pdb_data = f.read()

view = py3Dmol.view(width=600, height=400)
view.addModel(pdb_data, "pdb")
view.setStyle({"cartoon": {"color": "spectrum"}})
view.zoomTo()
view.show()

PDB file saved to: 4I22.pdb


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [10]:
# Style protein as cartoon colored by chain
view.setStyle({'chain': 'A'}, {'cartoon': {'color': 'spectrum'}})

# Style ligand(s) as sticks and spheres (heteroatoms)
#view.setStyle({'hetero': True}, {'stick': {}, 'sphere': {'radius': 0.3, 'opacity': 0.7}})
view.setStyle({'resn': 'IRE'}, {'stick': {}, 'sphere': {'radius': 0.3}})
#view.setStyle(
#    {'hetflag': True, 'resn': ['HOH', 'WAT'], 'invert': True},
#    {'stick': {}, 'sphere': {'radius': 0.3, 'opacity': 0.7}}
#)

view.zoomTo()
view.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [42]:
from Bio import PDB
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio import SeqIO

# Function to convert 3-letter to 1-letter amino acid code
aa_three_to_one = {
    'Ala': 'A', 'Arg': 'R', 'Asn': 'N', 'Asp': 'D', 'Cys': 'C',
    'Gln': 'Q', 'Glu': 'E', 'Gly': 'G', 'His': 'H', 'Ile': 'I',
    'Leu': 'L', 'Lys': 'K', 'Met': 'M', 'Phe': 'F', 'Pro': 'P',
    'Ser': 'S', 'Thr': 'T', 'Trp': 'W', 'Tyr': 'Y', 'Val': 'V',
    'Ter': '*'
}

def parser_pdb2seq(pdb_file):
    # Create parser
    parser = PDB.PDBParser(QUIET=True)
    structure = parser.get_structure("protein", pdb_file)

    sequences = []

    for model in structure:
        for chain in model:
            residues = []
            for res in chain:
                if PDB.is_aa(res, standard=True):
                    try:
                        residues.append(aa_three_to_one[res.get_resname().capitalize()])
                    except KeyError:
                        residues.append('X')  # Unknown residue
            seq_str = ''.join(residues)
            seq_record = SeqRecord(Seq(seq_str), id=f"{structure.id}_{chain.id}", description="")
            sequences.append(seq_record)
    
    # Print sequences for all chains
    for seq_record in sequences:
        print(f">{seq_record.id}")
        print(seq_record.seq)
        print(len(seq_record.seq))
    return seq_record.seq

def check_missing_residue(pdb_file): 

    parser = PDB.PDBParser(QUIET=True)
    structure = parser.get_structure("prot", pdb_file)

    # Read header to find missing residues
    with open(pdb_file) as f:
        lines = f.readlines()

    missing_residues = []
    for line in lines:
        if line.startswith("REMARK 465"):
            #print(len(line.split()))
            #missing_residues.append(line.strip())
            if line.split()[-2]=='A':
                missing_residues.append(line.split()[-1])

    print("Missing residues (from REMARK 465):")
    #for mr in missing_residues:
    #    print(mr)
    missing_residues = list(map(int, missing_residues))
    print(missing_residues)
    return missing_residues

def check_missing_indexes(list1, list2):
    indexes = [list2.index(x) if x in list2 else None for x in list1]
    #print(indexes)
    return indexes

def drop_by_index(mylist, drop_indexes):
    filtered = [x for i, x in enumerate(mylist) if i not in drop_indexes]
    return filtered

from Bio import pairwise2
from Bio.pairwise2 import format_alignment

# Path to your PDB file
pdb_id = "4I22"
exam_pdb = "./tkd_variants_pdb/p.Cys797Ser"
pdb_file1 = f"{pdb_id}.pdb"
pdb_file2 = f"{exam_pdb}.pdb"

seq1 = parser_pdb2seq(pdb_file1)
seq2 = parser_pdb2seq(pdb_file2)

# Local alignment (Smith-Waterman)
alignments = pairwise2.align.localms(seq1, seq2, 2, -1, -0.5, -0.1)
# Best alignment
best = alignments[0]
print(format_alignment(*best))
aligned_seq1, aligned_seq2, score, start, end = best


start1 = 700
start2 = 1
# Extract aligned indices
idx1, idx2 = [], []
pos1, pos2 = start1, start2
for a, b in zip(aligned_seq1, aligned_seq2):
    if a != "-":
        pos1 += 1
    if b != "-":
        pos2 += 1
    if a != "-" and b != "-":
        idx1.append(pos1 - 1)  # 0-based index
        idx2.append(pos2 - 1)
'''
print("length of indices in seq1:", len(idx1))
print("Indices in seq1:", idx1)
print("length of indices in seq2:", len(idx2))
print("Indices in seq2:", idx2)
'''
missing_residues = check_missing_residue(pdb_file1)
missing_indexes = check_missing_indexes(missing_residues, idx1)
#print("missing_indexes: ",missing_indexes)
ref_atoms =  drop_by_index(idx1, missing_indexes)
sample_atoms = drop_by_index(idx2, missing_indexes)
'''
print("length of indices in ref_atoms:", len(ref_atoms))
print("Indices in ref_atoms:", ref_atoms)
print("length of indices in sample_atoms:", len(sample_atoms))
print("Indices in sample_atoms:", sample_atoms)
'''

>protein_A
NQALLRILKETEFKKIKVLGSGAFGTVYKGLWIPEGEKVKIPVAIKELRKANKEILDEAYVMASVDNPHVCRLLGICLTSTVQLIMQLMPFGCLLDYVREHKDNIGSQYLLNWCVQIAKGMNYLEDRRLVHRDLAARNVLVKTPQHVKITDFGRAKLLVPIKWMALESILHRIYTHQSDVWSYGVTVWELMTFGSKPYDGIPASEISSILEKGERLPQPPICTIDVYMIMRKCWMIDADSRPKFRELIIEFSKMARDPQRYLVIQGDERMHLPSPTDSNFYRALMDEEDMDDVVDAD
297
>protein_A
FKKIKVLGSGAFGTVYKGLWIPEGEKVKIPVAIKELREATSPKANKEILDEAYVMASVDNPHVCRLLGICLTSTVQLITQLMPFGSLLDYVREHKDNIGSQYLLNWCVQIAKGMNYLEDRRLVHRDLAARNVLVKTPQHVKITDFGLAKLLGAEEKEYHAEGGKVPIKWMALESILHRIYTHQSDVWSYGVTVWELMTFGSKPYDGIPASEISSILEKGERLPQPPICTIDVYMIMVKCWMIDADSRPKFRELIIEFSKMARDPQRY
267
13 FKKIKVLGSGAFGTVYKGLWIPEGEKVKIPVAIKELR-----KANKEILDEAYVMASVDNPHVCRLLGICLTSTVQLIM-QLMPFGC-LLDYVREHKDNIGSQYLLNWCVQIAKGMNYLEDRRLVHRDLAARNVLVKTPQHVKITDFGR-AKLL-------------VPIKWMALESILHRIYTHQSDVWSYGVTVWELMTFGSKPYDGIPASEISSILEKGERLPQPPICTIDVYMIMR-KCWMIDADSRPKFRELIIEFSKMARDPQRY
   |||||||||||||||||||||||||||||||||||||     ||||||||||||||||||||||||||||||||||||  ||||||  ||||||||||||||||||||||||||||||||||||||

'\nprint("length of indices in ref_atoms:", len(ref_atoms))\nprint("Indices in ref_atoms:", ref_atoms)\nprint("length of indices in sample_atoms:", len(sample_atoms))\nprint("Indices in sample_atoms:", sample_atoms)\n'

In [ ]:
with open(pdb_file1, "r") as f:
    pdb_data = f.read()
view = py3Dmol.view(width=600, height=400)
view.addModel(pdb_data, "pdb")
# Style protein as cartoon colored by chain
view.setStyle({}, {"cartoon": {"color": "lightgrey", "opacity": 0.5}})
view.setStyle({'resi': list(range(749,858))}, {'cartoon': {'color': 'spectrum'}})
# Style ligand(s) as sticks and spheres (heteroatoms)
#view.setStyle({'hetero': True}, {'stick': {}, 'sphere': {'radius': 0.3, 'opacity': 0.7}})
view.setStyle({'resn': 'IRE'}, {'stick': {}, 'sphere': {'radius': 0.3}})
#view.setStyle(
#    {'hetflag': True, 'resn': ['HOH', 'WAT'], 'invert': True},
#    {'stick': {}, 'sphere': {'radius': 0.3, 'opacity': 0.7}}
#)

view.zoomTo()
view.show()

In [ ]:
with open(pdb_file2, "r") as f:
    pdb_data = f.read()
view = py3Dmol.view(width=600, height=400)
view.addModel(pdb_data, "pdb")
# Style protein as cartoon colored by chain
view.setStyle({}, {"cartoon": {"color": "lightgrey", "opacity": 0.5}})
view.setStyle({'resi': list(range(42,151))}, {'cartoon': {'color': 'spectrum'}})
# Style ligand(s) as sticks and spheres (heteroatoms)
#view.setStyle({'hetero': True}, {'stick': {}, 'sphere': {'radius': 0.3, 'opacity': 0.7}})
#view.setStyle({'resn': 'IRE'}, {'stick': {}, 'sphere': {'radius': 0.3}})
#view.setStyle(
#    {'hetflag': True, 'resn': ['HOH', 'WAT'], 'invert': True},
#    {'stick': {}, 'sphere': {'radius': 0.3, 'opacity': 0.7}}
#)

view.zoomTo()
view.show()

In [37]:
import Bio.PDB

def pdb_align(ref_atoms_to_be_aligned, ref, sample_atoms_to_be_aligned, sample):
    # Select what residues numbers you wish to align
    # and put them in a list
    #start_id = 1
    #end_id   = 70
    #ref_atoms_to_be_aligned = range(ref_start, ref_end + 1)
    #sample_atoms_to_be_aligned = range(sample_start, sample_end + 1)
    print(len(ref_atoms_to_be_aligned))
    print(len(sample_atoms_to_be_aligned))
    # Start the parser
    pdb_parser = Bio.PDB.PDBParser(QUIET = True)

    # Get the structures
    #ref_structure = pdb_parser.get_structure("reference", ref+".pdb")
    ref_structure = pdb_parser.get_structure("reference", ref)
    #sample_structure = pdb_parser.get_structure("sample", sample+".pdb")
    sample_structure = pdb_parser.get_structure("sample", sample)
    #sample_structure = Sructure(sample)

    # Use the first model in the pdb-files for alignment
    # Change the number 0 if you want to align to another structure
    ref_model    = ref_structure[0]
    sample_model = sample_structure[0]

    # Make a list of the atoms (in the structures) you wish to align.
    # In this case we use CA atoms whose index is in the specified range
    ref_atoms = []
    sample_atoms = []

    # Iterate of all chains in the model in order to find all residues
    for ref_chain in ref_model:
        print(ref_chain.id)
        if ref_chain.id == 'A':
            # Iterate of all residues in each model in order to find proper atoms
            for ref_res in ref_chain:
                # Check if residue number ( .get_id() ) is in the list
                #print(ref_res.get_id()[1])
                if ref_res.get_id()[1] in ref_atoms_to_be_aligned:
                    # Append CA atom to list
                    ref_atoms.append(ref_res['CA'])

    # Do the same for the sample structure
    for sample_chain in sample_model:
        for sample_res in sample_chain:
            if sample_res.get_id()[1] in sample_atoms_to_be_aligned:
                sample_atoms.append(sample_res['CA'])
    print(len(ref_atoms))
    print(len(sample_atoms))

    # Now we initiate the superimposer:
    super_imposer = Bio.PDB.Superimposer()
    super_imposer.set_atoms(ref_atoms, sample_atoms)
    super_imposer.apply(sample_model.get_atoms())

    # Print RMSD:
    print(super_imposer.rms)

    # Save the aligned version of 1UBQ.pdb
    #io = Bio.PDB.PDBIO()
    #io.set_structure(sample_structure)
    #io.save("1UBQ_aligned.pdb")
    return sample_structure


In [65]:
from pathlib import Path
#print(ref_atoms)
#print(sample_atoms)
## align sample_protein to ref_protien using the first 100 residues that are around the binding pocket to get a better result.
align_structure=pdb_align(ref_atoms[:50], pdb_file1,sample_atoms[:50], pdb_file2)
io = Bio.PDB.PDBIO()
io.set_structure(align_structure)
outputfolder='tkd_variants_pdb'
io.save(outputfolder+'/'+Path(pdb_file2).stem+"_aligned_temp.pdb")

50
50
A
50
50
3.1782621843282155


In [74]:
import py3Dmol
## apo state
with open(r"./tkd_variants_pdb/p.Cys797Ser_aligned_temp.pdb", "r") as f:
    protein = f.read()

with open(r"./4I22.pdb", "r") as f:
    ligand = f.read()
    
view = py3Dmol.view(width=600, height=400)
view.addModel(protein, "pdb")

view.setStyle({'cartoon': {'color':'lightgrey'}})
view.setStyle({'chain':'A','resi':list(range(0, 100))},{'cartoon': {'color':'spectrum'}})
view.addModel(ligand, "pdb")
#view.setStyle({'cartoon': {'color':'lightgrey'}})

view.setStyle({'resn': 'IRE'}, {'stick': {'colorscheme': 'greenCarbon'}})
view.addSurface(py3Dmol.VDW, {'opacity': 0.3, 'color': 'yellow'}, {'resn': 'LIG'})

view.zoomTo()
view.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [57]:
from Bio.PDB import PDBParser
import numpy as np

def get_ligand_center(pdb_file, ligand_resname="IRE"):
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure("ligand", pdb_file)

    coords = []
    for atom in structure.get_atoms():
        if atom.get_parent().get_resname() == ligand_resname:
            coords.append(atom.coord)

    coords = np.array(coords)
    center = coords.mean(axis=0)
    size = coords.max(axis=0) - coords.min(axis=0)

    return center, size

center, size = get_ligand_center("4I22.pdb", "IRE")
print("Center:", center)
print("Box size:", size + 5)  # Add padding (~5Å)

Center: [ 10.199838 -15.762483  11.211613]
Box size: [17.609    15.188001 16.129002]


In [59]:
### the p.Cys797Ser_aligned was aligned with 4I22.pdb that has a IRE ligand binding
### the docking task here is to dock drugs at this binding pocket
### the simple approach is to move COM of drug to the COM of IRE 

from Bio.PDB import PDBParser, PDBIO
import numpy as np

# Input parameters
pdb_file = "./tkd_variants_pdb/p.Cys797Ser_aligned_temp.pdb"
output_file = "./tkd_variants_pdb/p.Cys797Ser_aligned.pdb"
chain_to_move = "A"
shift_array = center  # x, y, z shift

# Parse PDB
parser = PDBParser(QUIET=True)
structure = parser.get_structure("prot", pdb_file)

# Apply translation
for model in structure:
    for chain in model:
        if chain.id == chain_to_move:
            for residue in chain:
                for atom in residue:
                    atom.coord = atom.coord - shift_array

# Save the new structure
io = PDBIO()
io.set_structure(structure)
io.save(output_file)

print(f"Chain {chain_to_move} moved by {shift_array} and saved to {output_file}")


Chain A moved by [ 10.199838 -15.762483  11.211613] and saved to ./tkd_variants_pdb/p.Cys797Ser_aligned.pdb


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [63]:
from Bio.PDB import PDBParser, PDBIO
import numpy as np

pdb_file = "4I22.pdb"
output_file = "ligand_centered.pdb"
ligand_resname = "IRE"   # change to your ligand's residue name

# Parse structure
parser = PDBParser(QUIET=True)
structure = parser.get_structure("complex", pdb_file)

# Find ligand atoms
ligand_atoms = []
for model in structure:
    for chain in model:
        for residue in chain:
            if residue.get_resname().strip() == ligand_resname:
                ligand_atoms.extend(list(residue.get_atoms()))

if not ligand_atoms:
    raise ValueError(f"Ligand with resname {ligand_resname} not found!")

# Compute center of mass (geometric, equal weights)
coords = np.array([atom.coord for atom in ligand_atoms])
com = coords.mean(axis=0)

print(f"Original COM of {ligand_resname}: {com}")

# Shift vector = -COM
shift = -com

# Apply translation to ligand atoms only
for atom in ligand_atoms:
    atom.coord = atom.coord + shift

# Save new structure
io = PDBIO()
io.set_structure(structure)
io.save(output_file)

print(f"Ligand {ligand_resname} moved so its COM is at [0,0,0] -> saved to {output_file}")

Original COM of IRE: [ 10.199838 -15.762483  11.211613]
Ligand IRE moved so its COM is at [0,0,0] -> saved to ligand_centered.pdb


In [64]:
import py3Dmol
## apo state
with open(r"./tkd_variants_pdb/p.Cys797Ser_aligned.pdb", "r") as f:
    protein = f.read()

with open(r"./ligand_centered.pdb", "r") as f:
    ligand = f.read()
    
view = py3Dmol.view(width=600, height=400)
view.addModel(protein, "pdb")  
view.setStyle({'cartoon': {'color':'spectrum'}})

view.addModel(ligand, "pdb")
#view.setStyle({'cartoon': {'color':'spectrum'}})

view.setStyle({'resn': 'IRE'}, {'stick': {'colorscheme': 'greenCarbon'}})
view.addSurface(py3Dmol.VDW, {'opacity': 0.3, 'color': 'yellow'}, {'resn': 'LIG'})

view.zoomTo()
view.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.